In [6]:
import pandas as pd
import nltk
import gensim
import numpy as np
df = pd.read_csv('./content/archive/20000-Utterances-Training-dataset-for-chatbots-virtual-assistant-Bitext-sample/20000-Utterances-Training-dataset-for-chatbots-virtual-assistant-Bitext-sample/20000-Utterances-Training-dataset-for-chatbots-virtual-assistant-Bitext-sample.csv')
df.head(5)

,flags,utterance,category,intent
0,BILC,"I don't have an online account, what do I have...",ACCOUNT,create_account
1,BILQZ,can you tell me if i can regisger two accounts...,ACCOUNT,create_account
2,BPLC,"I have no online account, open one, please",ACCOUNT,create_account
3,BIPLD,"could you ask an agent how to open an account,...",ACCOUNT,create_account
4,BLQC,"i want an online account, create one",ACCOUNT,create_account


# Basic Processing

In [7]:
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
english_stopwords = stopwords.words('english')
porter = PorterStemmer()
def preprocess(text):
    text = text.lower()
    token = word_tokenize(text)
    token = [porter.stem(word) for word in token if (word not in english_stopwords) and word.isalpha()]
    return token
df['words'] = df['utterance'].apply(lambda x: preprocess(x))
print(df['words'].head())
print(df['words'].describe())

0                             [onlin, account, regist]
1    [tell, regisg, two, account, singl, email, add...
2                   [onlin, account, open, one, pleas]
3            [could, ask, agent, open, account, pleas]
4                   [want, onlin, account, creat, one]
Name: words, dtype: object
count           21534
unique          10836
top       [see, bill]
freq               31
Name: words, dtype: object


In [8]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.2, random_state = 1337)

# Building Embedding Layer

In [9]:
import pandas as pd
import numpy as np
import gensim
import tensorflow as tf

In [10]:
# Download from https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?resourcekey=0-wjGZdNAUop6WykTtMip30g
import gensim.downloader as api
wv_pretrain = api.load('word2vec-google-news-300')

In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_train['words'])
encoded_train_docs = tokenizer.texts_to_sequences(df_train['words'])
encoded_test_docs = tokenizer.texts_to_sequences(df_test['words'])
max_length = max([len(s) for s in df_train['words']])
X_train = pad_sequences(encoded_train_docs, 
                        maxlen=max_length, 
                        padding='post')
y_train = pd.get_dummies(df_train['category'])
X_test = pad_sequences(encoded_test_docs, 
                        maxlen=max_length, 
                        padding='post')
y_test = pd.get_dummies(df_test['category'])

In [12]:
# word2vec + cnn
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding, Conv1D, MaxPooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

def get_word2vec_embed_layer(max_length, tokenizer, model):
    word_index = tokenizer.word_index
    embedding_mat = np.zeros((len(word_index)+1, 300))
    for word, i in word_index.items():
        try:
            vector = model.get_vector(word)
            embedding_mat[i] = vector
        except:
            continue
    word2vec_embedding_layer = Embedding(input_dim=embedding_mat.shape[0],
                                      output_dim=embedding_mat.shape[1], 
                                      weights=[embedding_mat],
                                      input_length=max_length, 
                                      trainable=False)
    return word2vec_embedding_layer

embedding = get_word2vec_embed_layer(X_train.shape[1], tokenizer, w2v_pretrain)

In [13]:
y_train_arr = np.array(y_train).reshape(-1,1,y_train.shape[1])
y_test_arr = np.array(y_test).reshape(-1,1,y_test.shape[1])

# RNN Model for topic detection

In [14]:
from tensorflow.keras.layers import LSTM, GRU, Dense
from tensorflow.keras.callbacks import EarlyStopping

In [15]:
model = Sequential()
model.add(embedding)

model.add(LSTM(64))
model.add(Dense(y_train.shape[1],activation='softmax'))
callback = EarlyStopping(monitor='val_loss', patience=3)

model.compile(loss='CategoricalCrossentropy',metrics=['Accuracy'])

Metal device set to: Apple M1


2022-08-06 14:35:53.308274: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-06 14:35:53.309986: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [16]:
model.fit(X_train,y_train,validation_split=0.1,epochs=100,callbacks=[callback])

Epoch 1/100


2022-08-06 14:35:53.751989: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-08-06 14:35:54.610875: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-06 14:35:54.943934: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-06 14:35:55.652248: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


485/485 [==============================] - ETA: 0s - loss: 0.4304 - Accuracy: 0.8700

2022-08-06 14:36:01.254499: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-06 14:36:01.298651: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


485/485 [==============================] - 8s 12ms/step - loss: 0.4304 - Accuracy: 0.8700 - val_loss: 0.1819 - val_Accuracy: 0.9437
Epoch 2/100
485/485 [==============================] - 6s 12ms/step - loss: 0.1153 - Accuracy: 0.9618 - val_loss: 0.1084 - val_Accuracy: 0.9605
Epoch 3/100
485/485 [==============================] - 6s 12ms/step - loss: 0.0873 - Accuracy: 0.9688 - val_loss: 0.0839 - val_Accuracy: 0.9698
Epoch 4/100
485/485 [==============================] - 5s 11ms/step - loss: 0.0763 - Accuracy: 0.9716 - val_loss: 0.0787 - val_Accuracy: 0.9698
Epoch 5/100
485/485 [==============================] - 6s 12ms/step - loss: 0.0695 - Accuracy: 0.9728 - val_loss: 0.0769 - val_Accuracy: 0.9692
Epoch 6/100
485/485 [==============================] - 5s 11ms/step - loss: 0.0662 - Accuracy: 0.9744 - val_loss: 0.0729 - val_Accuracy: 0.9716
Epoch 7/100
485/485 [==============================] - 5s 11ms/step - loss: 0.0622 - Accuracy: 0.9753 - val_loss: 0.0710 - val_Accuracy: 0.9756
Epoc

# Model Inference

In [17]:
model.save("RNN_chatbot")

INFO:tensorflow:Assets written to: RNN_chatbot/assets


INFO:tensorflow:Assets written to: RNN_chatbot/assets


In [18]:
topic_mapping =dict(zip(range(11),list(y_train.columns)))
print('Maxlen of prediction: ',max_length) # The incoming texts will be pad/ cut to length 13
topic_mapping

Maxlen of prediction:  13


{0: 'ACCOUNT',
 1: 'CANCELLATION_FEE',
 2: 'CONTACT',
 3: 'DELIVERY',
 4: 'FEEDBACK',
 5: 'INVOICES',
 6: 'NEWSLETTER',
 7: 'ORDER',
 8: 'PAYMENT',
 9: 'REFUNDS',
 10: 'SHIPPING'}

In [19]:
def accuracy_rate(df):
    return np.sum(df['True']!=df['Predict'])/len(df)
def model_evaluation(model,X,y):
    # Pass a list of word, label
    X_preprocessed = list(map(preprocess,X))
    X_input = pad_sequences(tokenizer.texts_to_sequences(X),maxlen=max_length,padding='post')
    y_pred = model.predict(X_input)
    class_prob = np.max(y_pred,axis=1)
    label = np.argmax(y_pred,axis=1)
    result = pd.DataFrame({'Word':X,'True':y,'Prob':class_prob,'Predict':label})
    result['Predict']= result['Predict'].apply(lambda x: topic_mapping[x])
    print('Accuracy: ',np.sum(result['True']==result['Predict'])/len(X))
    print('Top 5 misclassification:')
    temp = result[result['True']!=result['Predict']].sort_values(by='Prob').head(5)
    print(temp.to_string())
    print("-"*100)
    print("Misclassification Rate by category:")
    temp = result.groupby('True').apply(lambda x: accuracy_rate(x))
    print(temp)
    return result

## Evaluation on Training Set
The overall accuracy is quite satisfactory
The performance on some minority catrgories like Shipping,  Delivery,  cancellation_fee is quite bad subjucted to small sample size

In [20]:
result =model_evaluation(model,df_train['utterance'],df_train['category'])

 12/539 [..............................] - ETA: 2s  

2022-08-06 14:37:47.385649: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-06 14:37:47.431501: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


539/539 [==============================] - 2s 3ms/step
Accuracy:  0.8096012074069774
Top 5 misclassification:
                                             Word      True      Prob   Predict
1052                       ho to rceate a profile   ACCOUNT  0.224204  DELIVERY
8397            how sono can i expect my tickets?  DELIVERY  0.238577  INVOICES
3914       my profile was hacked, can u belp me ?   ACCOUNT  0.263164   REFUNDS
7463                            u are not helping   CONTACT  0.263164   REFUNDS
3876  my account's been hacked, can u recover it?   ACCOUNT  0.263164   REFUNDS
----------------------------------------------------------------------------------------------------
Misclassification Rate by category:
True
ACCOUNT             0.240000
CANCELLATION_FEE    0.736842
CONTACT             0.069303
DELIVERY            0.415423
FEEDBACK            0.039305
INVOICES            0.206582
NEWSLETTER          0.175141
ORDER               0.262688
PAYMENT             0.153887
REFUNDS

## Evalution on Test Set
The performance on testing set is quite consistent with the one in training set, indicate there is no sign of overfitting

In [21]:
result =model_evaluation(model,df_test['utterance'],df_test['category'])

135/135 [==============================] - 0s 3ms/step
Accuracy:  0.802646853958672
Top 5 misclassification:
                                                                        Word     True      Prob  Predict
6442  I want information, I ned help speaking with Customer Servgice, please  CONTACT  0.234961  ACCOUNT
3202                                my profile was hacked, can u recover it?  ACCOUNT  0.263164  REFUNDS
7002                                                  can u contcat someone?  CONTACT  0.263164  REFUNDS
4266                                           I'm having problems signing u  ACCOUNT  0.263164  REFUNDS
3570                                my account's been hacked, can u help me?  ACCOUNT  0.278325  REFUNDS
----------------------------------------------------------------------------------------------------
Misclassification Rate by category:
True
ACCOUNT             0.236052
CANCELLATION_FEE    0.746667
CONTACT             0.076433
DELIVERY            0.505051
FEE

# Inference on some self prepare enquiry

In [22]:
Enquiry=['I want to open an account',"The hat quality is poor, can I return it?","Find somebody to talk with me or cotact me later"]
# Enquiry 1 and 2 should be some quite standard Account and Refund enquiry.
# Enquiry 3 is about contact, but there is some typo
topic_ans = ['ACCOUNT','REFUNDS','CONTACT']

In [23]:
model_evaluation(model,Enquiry,topic_ans)

1/1 [==============================] - 0s 91ms/step
Accuracy:  0.6666666666666666
Top 5 misclassification:
                                               Word     True      Prob   Predict
2  Find somebody to talk with me or cotact me later  CONTACT  0.626085  INVOICES
----------------------------------------------------------------------------------------------------
Misclassification Rate by category:
True
ACCOUNT    0.0
CONTACT    1.0
REFUNDS    0.0
dtype: float64


,Word,True,Prob,Predict
0,I want to open an account,ACCOUNT,0.999981,ACCOUNT
1,"The hat quality is poor, can I return it?",REFUNDS,0.549255,REFUNDS
2,Find somebody to talk with me or cotact me later,CONTACT,0.626085,INVOICES
